### 作业处理
1. 在 100 万例训练集上训练（没有限制）的决策树的近似深度是多少？
包含m个叶子的均衡二叉树的深度大概等于$log_2(m)^3$ 。如果训练集包含一百万个实例，则决策树将具有$log_2(10^6)≈20$的深度。

2. 节点的基尼指数比起它的父节点是更高还是更低？它是通常情况下更高/更低，还是永远更高/更低？
节点的Gini比父节点更低，这是通过CART训练算法的成本函数来确保的，该算法将每个节点分开，以最小化其子节点Gini杂质的加权和。  

3. 如果决策树过拟合了，减少最大深度是一个好的方法吗？
是的，可以通过约束模型，使模型正则化。

4. 如果决策树对训练集欠拟合了，尝试缩放输入特征是否是一个好主意？
决策树不关心训练数据是否按比例缩放或居中；对于决策树欠拟合训练集，缩放输入特征作用不大。

5. 如果对包含 100 万个实例的数据集训练决策树模型需要一个小时，在包含 1000 万个实例的培训集上训练另一个决策树大概需要多少时间呢？
训练决策树的计算复杂度是$O(n × m log(m))$，因此，如果将训练集大小乘以10，则训练时间将乘以$K = (n × 10m × log(10m)) / (n × m × log(m)) = 10 × log(10m) / log(m)$
如果m = 10^6，则K≈11.7，因此你可以预期训练时间大约为11.7小时。

6. 如果你的训练集包含 100,000 个实例，设置presort=True会加快训练的速度吗？
不会，教程中说是小于几千的时候做presort会加快速度。

7. 对moons数据集进行决策树训练并优化模型。

 - 通过语句make_moons(n_samples=10000, noise=0.4)生成moons数据集

 - 通过train_test_split()将数据集分割为训练集和测试集。

 - 进行交叉验证，并使用网格搜索法寻找最好的超参数值（使用GridSearchCV类的帮助文档）  
**提示: 尝试各种各样的max_leaf_nodes值。使用这些超参数训练全部的训练集数据，并在测试集上测量模型的表现。你应该获得大约 85% 到 87% 的准确度。**
~~~python
from sklearn.datasets import make_moons
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
#获取数据
X,y=make_moons(n_samples=10000, noise=0.4)
#分割数据
X_train,X_test, y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
使用网格搜索进行交叉验证，找出好的超参数，注意里面params参数设定
params={
    "max_leaf_nodes":list(range(2,100)),
    "min_samples_split":[2,3,4]
}
grid_search_cv=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1,verbose=1,cv=3)
grid_search_cv.fit(X_train,y_train)
#训练结果
'''
Fitting 3 folds for each of 294 candidates, totalling 882 fits
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:   16.3s finished
GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 'min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)
'''
#正确率86%
y_pred=grid_search_cv.predict(X_test)
accuracy_score(y_test,y_pred)
~~~

8. 生成森林
 - 接着前边的练习，现在，让我们生成 1,000 个训练集的子集，每个子集包含 100 个随机选择的实例。提示：你可以使用 Scikit-Learn 的ShuffleSplit类。
 - 使用上面找到的最佳超参数值，在每个子集上训练一个决策树。在测试集上测试这 1000 个决策树。由于它们是在较小的集合上进行了训练，因此这些决策树可能会比第一个决策树效果更差，只能达到约 80% 的准确度。
 - 见证奇迹的时刻到了！对于每个测试集实例，生成 1,000 个决策树的预测结果，然后只保留出现次数最多的预测结果（您可以使用 SciPy 的mode()函数）。这个函数使你可以对测试集进行多数投票预测。
 - 在测试集上评估这些预测结果，你应该获得了一个比第一个模型高一点的准确率，（大约 0.5% 到 1.5%），恭喜，你已经弄出了一个随机森林分类器模型!
~~~python

from sklearn.model_selection import ShuffleSplit
#生成 1,000 个训练集的子集，每个子集包含 100 个随机选择的实例。提示：你可以使用 Scikit-Learn 的ShuffleSplit类。
n_tree=1000
n_instance=1000
mini_sets=[]
rs=ShuffleSplit(n_splits=n_tree,test_size=len(X_train)-n_instance,random_state=42)
for mini_train_index ,mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))
    
from sklearn.base import clone
import numpy as np
#使用上面找到的最佳超参数值，在每个子集上训练一个决策树。在测试集上测试这 1000 个决策树。由于它们是在较小的集合上进行了训练，因此这些决策树可能会比第一个决策树效果更差，只能达到约 80% 的准确度。
forest=[clone(grid_search_cv.best_estimator_)for _ in range(n_tree)]
accuracy_scores=[]
for tree, (X_mini_train,y_mini_train) in zip(forest,mini_sets):
    tree.fit(X_mini_train,y_mini_train)
    y_pred=tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test,y_pred))
np.mean(accuracy_scores)

# 见证奇迹的时刻到了！对于每个测试集实例，生成 1,000 个决策树的预测结果，然后只保留出现次数最多的预测结果（您可以使用 SciPy 的mode()函数）。这个函数使你可以对测试集进行多数投票预测。
Y_pred=np.empty([n_tree,len(X_test)],dtype=np.uint8)
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index]=tree.predict(X_test)

from scipy.stats import mode
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

#在测试集上评估这些预测结果，你应该获得了一个比第一个模型高一点的准确率，（大约 0.5% 到 1.5%），恭喜，你已经弄出了一个随机森林分类器模型!
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))
~~~

In [8]:
from sklearn.datasets import make_moons
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
X,y=make_moons(n_samples=10000, noise=0.4)
X_train,X_test, y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
params={
    "max_leaf_nodes":list(range(2,100)),
    "min_samples_split":[2,3,4]
}
grid_search_cv=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1,verbose=1,cv=3)
grid_search_cv.fit(X_train,y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:   16.3s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 'min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, v

In [9]:
from sklearn.metrics import accuracy_score
y_pred=grid_search_cv.predict(X_test)
accuracy_score(y_test,y_pred)

0.856

8. 生成森林
 - 接着前边的练习，现在，让我们生成 1,000 个训练集的子集，每个子集包含 100 个随机选择的实例。提示：你可以使用 Scikit-Learn 的ShuffleSplit类。
 - 使用上面找到的最佳超参数值，在每个子集上训练一个决策树。在测试集上测试这 1000 个决策树。由于它们是在较小的集合上进行了训练，因此这些决策树可能会比第一个决策树效果更差，只能达到约 80% 的准确度。
 - 见证奇迹的时刻到了！对于每个测试集实例，生成 1,000 个决策树的预测结果，然后只保留出现次数最多的预测结果（您可以使用 SciPy 的mode()函数）。这个函数使你可以对测试集进行多数投票预测。
 - 在测试集上评估这些预测结果，你应该获得了一个比第一个模型高一点的准确率，（大约 0.5% 到 1.5%），恭喜，你已经弄出了一个随机森林分类器模型!

In [11]:
from sklearn.model_selection import ShuffleSplit
#生成 1,000 个训练集的子集，每个子集包含 100 个随机选择的实例。提示：你可以使用 Scikit-Learn 的ShuffleSplit类。
n_tree=1000
n_instance=1000
mini_sets=[]
rs=ShuffleSplit(n_splits=n_tree,test_size=len(X_train)-n_instance,random_state=42)
for mini_train_index ,mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [17]:
from sklearn.base import clone
import numpy as np
#使用上面找到的最佳超参数值，在每个子集上训练一个决策树。在测试集上测试这 1000 个决策树。由于它们是在较小的集合上进行了训练，因此这些决策树可能会比第一个决策树效果更差，只能达到约 80% 的准确度。
forest=[clone(grid_search_cv.best_estimator_)for _ in range(n_tree)]
accuracy_scores=[]
for tree, (X_mini_train,y_mini_train) in zip(forest,mini_sets):
    tree.fit(X_mini_train,y_mini_train)
    y_pred=tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test,y_pred))
np.mean(accuracy_scores)

0.8450045

In [20]:
# 见证奇迹的时刻到了！对于每个测试集实例，生成 1,000 个决策树的预测结果，然后只保留出现次数最多的预测结果（您可以使用 SciPy 的mode()函数）。这个函数使你可以对测试集进行多数投票预测。
Y_pred=np.empty([n_tree,len(X_test)],dtype=np.uint8)
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index]=tree.predict(X_test)


In [21]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [22]:
#在测试集上评估这些预测结果，你应该获得了一个比第一个模型高一点的准确率，（大约 0.5% 到 1.5%），恭喜，你已经弄出了一个随机森林分类器模型!
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.858